# Classifcation of MNIST data using pytorch

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np
from torchvision import datasets, transforms

In [58]:
transform = transforms.Compose([transforms.ToTensor()])

mnist_pytorch = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

train_loader_pytorch = torch.utils.data.DataLoader(mnist_pytorch, batch_size=1, shuffle=False)



In [59]:
mnist_pytorch_train_data = mnist_pytorch.data.view(mnist_pytorch_train_data.size(0), -1)
mnist_pytorch_train_labels = mnist_pytorch.targets.long()

print(mnist_pytorch_train_labels.shape)

torch.Size([60000])


In [54]:
class ClassifcationNN(nn.Module):
    '''
    Class for a feed forward neural network to classify MNIST data 
    '''
    def __init__(self, input_size : int = 784, layer_1_size:int = 800, output_size : int = 10):
        super(ClassifcationNN, self).__init__()
        self.input_layer = nn.Linear(input_size, layer_1_size)
        self.first_layer = nn.Linear(layer_1_size, output_size)

        self.ReLU = nn.ReLU()
        self.softmax = nn.Softmax()

    def forward(self, input_data : np.ndarray) -> np.ndarray:
        # Forward propagation
        out = self.input_layer(input_data)
        out = self.ReLU(out)
        out = self.first_layer(out)
        out = self.softmax(out)
        return out
    
    
def train(input_data : np.ndarray, target_data : np.ndarray , model : nn.Module, num_epoches :int = 5000) -> np.ndarray:
    # Function for training a model
    criterion = nn.CrossEntropyLoss()
    optimize = optim.Adam(model.parameters(),lr = 0.001 )

    for _ in range(num_epoches):
        model.train()
        # Forward pass
        outputs = model(input_data)
        loss = criterion(target_data, input_data)
        
        # Backward pass and optimization
        optimize.zero_grad()  # Clear gradients
        loss.backward()  # Backpropagation
        optimize.step()  # Update weights
        
    return model
   

In [52]:
trained_model = train(mnist_pytorch_train_data, mnist_pytorch_train_labels, ClassifcationNN())

RuntimeError: mat1 and mat2 must have the same dtype, but got Byte and Float